In [1]:
from source import TheData as W

w = W()
w

In [11]:
w.df

{2000: Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
             ...
             1286, 1287, 1288, 1289, 1290, 1291, 1292, 1293, 1294, 1295],
            dtype='int64', length=1296),
 2001: Int64Index([1296, 1297, 1298, 1299, 1300, 1301, 1302, 1303, 1304, 1305,
             ...
             2581, 2582, 2583, 2584, 2585, 2586, 2587, 2588, 2589, 2590],
            dtype='int64', length=1295),
 2002: Int64Index([2591, 2592, 2593, 2594, 2595, 2596, 2597, 2598, 2599, 2600,
             ...
             3876, 3877, 3878, 3879, 3880, 3881, 3882, 3883, 3884, 3885],
            dtype='int64', length=1295),
 2003: Int64Index([3886, 3887, 3888, 3889, 3890, 3891, 3892, 3893, 3894, 3895,
             ...
             5169, 5170, 5171, 5172, 5173, 5174, 5175, 5176, 5177, 5178],
            dtype='int64', length=1293),
 2004: Int64Index([5179, 5180, 5181, 5182, 5183, 5184, 5185, 5186, 5187, 5188,
             ...
             6466, 6467, 6468, 6469, 6470, 6471, 6472, 6473,